<h1>삼성 주식 예측 (RNN/LSTM)</h2>

<h2>라이브러리</h2>

In [11]:
#인공지능 모델이 기억을 가지고 있음
import pandas as pd
import numpy as np#행렬 연산
import matplotlib.pyplot as plt #데이터 시각화
from keras.models import Sequential# 딥러닝 모델
from keras.layers import LSTM, Dropout, Dense, Activation

<h2>데이터셋 가져오기</h2>

In [88]:
data=pd.read_csv('SAM.csv')
data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2015-08-25,210.020004,212.000000,202.419998,202.610001,202.610001,126900
1,2015-08-26,205.880005,205.940002,198.410004,205.149994,205.149994,280300
2,2015-08-27,205.589996,208.610001,204.279999,208.190002,208.190002,143600
3,2015-08-28,208.179993,209.990005,206.770004,208.419998,208.419998,85400
4,2015-08-31,208.020004,208.960007,204.009995,205.070007,205.070007,97200


<h2> Mid Price 계산</h2>

In [89]:
high_prices = data['High'].values
low_prices =data['Low'].values
mid_prices =(high_prices+low_prices)/2
mid_prices

array([207.209999, 202.175003, 206.445   , ..., 850.5     , 870.125   ,
       884.700012])

<h2>Ctreate Windows</h2>

In [90]:
seq_len=50 # 최근 50일 을 보고 다음 예측하겠다 (window size)
sequence_length=seq_len+1
#50개를 보고 1개를 예측 ( 51개 저장)
# 다음 윈도우는 한칸씩 밀려나면서 만들어짐 (0~51) 다음은 (1~51)

result =[]

for index in range(len(mid_prices)-sequence_length):
    result.append(mid_prices[index:index+sequence_length])
result
    #result에 51개 씩 한스텝씩 넘어가면서 저장

[array([207.209999 , 202.175003 , 206.445    , 208.3800045, 206.485001 ,
        199.7649995, 198.3149945, 205.3150025, 207.649994 , 207.514999 ,
        210.9199985, 214.639999 , 214.0649945, 221.814995 , 224.094994 ,
        224.4049985, 224.8700025, 225.660004 , 228.2400055, 225.134995 ,
        220.689995 , 215.9850005, 218.4800035, 216.040001 , 213.6199955,
        212.5599975, 209.6500015, 210.090004 , 216.209999 , 215.8650055,
        219.084999 , 222.380005 , 227.4850005, 227.6349945, 229.720001 ,
        230.0299985, 233.2249985, 239.8199995, 242.3099975, 252.1450045,
        256.830002 , 256.160004 , 257.409996 , 257.149994 , 252.529991 ,
        245.5849995, 247.5699995, 220.419998 , 223.300003 , 220.595001 ,
        217.0350035]),
 array([202.175003 , 206.445    , 208.3800045, 206.485001 , 199.7649995,
        198.3149945, 205.3150025, 207.649994 , 207.514999 , 210.9199985,
        214.639999 , 214.0649945, 221.814995 , 224.094994 , 224.4049985,
        224.8700025, 225.660

<h2>정규화</h2>

In [91]:
#첫번째 값을 0으로 기준으로 잡음
nomalized_data=[]
for window in result:
    nomalized_window = [((float(p)/float(window[0]))-1) for p in window]
    #첫번째값을 0으로 만들고 정규화하기
    nomalized_data.append(nomalized_window)

result=np.array(nomalized_data)
result

array([[ 0.00000000e+00, -2.42990011e-02, -3.69190195e-03, ...,
         7.76507122e-02,  6.45963132e-02,  4.74156872e-02],
       [ 0.00000000e+00,  2.11203014e-02,  3.06912398e-02, ...,
         9.11091763e-02,  7.35006815e-02,  5.42846462e-02],
       [ 0.00000000e+00,  9.37297828e-03,  1.93761050e-04, ...,
         5.12969726e-02,  3.24783913e-02,  7.99241444e-03],
       ...,
       [ 0.00000000e+00,  1.60020122e-02, -8.36339097e-03, ...,
         5.97003653e-01,  5.99568932e-01,  6.20205651e-01],
       [ 0.00000000e+00, -2.39816486e-02, -4.49315045e-02, ...,
         5.74375752e-01,  5.94687443e-01,  5.96583405e-01],
       [ 0.00000000e+00, -2.14646127e-02,  2.51767597e-02, ...,
         6.33870348e-01,  6.35812895e-01,  6.73558725e-01]])

In [92]:
#split train and test data 
row =int(round(result.shape[0]*0.9)) #(len(result))랑 같음
train=result[:row:]
np.random.shuffle(train)#배열값을 랜덤으로 셔플

x_train=train[:,:-1] 
x_train
x_train=np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))
y_train=train[:,-1]


In [96]:
x_test=result[row:,:-1]
x_test=np.reshape(x_test,(x_test.shape[0],x_test.shape[1],1))
y_test=result[row:,-1]

print(x_train.shape,x_test.shape)

(1087, 50, 1) (121, 50, 1)


<h2>Build a Model</h2>

In [98]:
model =Sequential()

model.add(LSTM(50,return_sequences=True,input_shape=(50,1)))

model.add(LSTM(64,return_sequences=False))  

model.add(Dense(1,activation='linear'))

model.compile(loss='mse',optimizer='rmsprop')

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 50, 50)            10400     
_________________________________________________________________
lstm_10 (LSTM)               (None, 64)                29440     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 39,905
Trainable params: 39,905
Non-trainable params: 0
_________________________________________________________________


<h2>Training</h2>

In [99]:
model.fit(x_train,y_train,
         validation_data=(x_test,y_test),
         batch_size=10,#한번에 몇개씩 묶어서 학습시킬것인가
         epochs=20) #반복학습 개수 

Train on 1087 samples, validate on 121 samples
Epoch 1/20
1087/1087 [==============================] - 11s 10ms/step - loss: 0.0028 - val_loss: 0.0041
Epoch 2/20
1087/1087 [==============================] - 9s 8ms/step - loss: 0.0012 - val_loss: 0.0046
Epoch 3/20
1087/1087 [==============================] - 9s 8ms/step - loss: 8.5900e-04 - val_loss: 0.0024
Epoch 4/20
1087/1087 [==============================] - 10s 9ms/step - loss: 6.9079e-04 - val_loss: 0.0021
Epoch 5/20
1087/1087 [==============================] - 10s 9ms/step - loss: 5.9128e-04 - val_loss: 0.0017
Epoch 6/20
1087/1087 [==============================] - 10s 9ms/step - loss: 5.4071e-04 - val_loss: 0.0016
Epoch 7/20
1087/1087 [==============================] - 10s 9ms/step - loss: 4.5946e-04 - val_loss: 0.0022
Epoch 8/20
1087/1087 [==============================] - 10s 9ms/step - loss: 4.4010e-04 - val_loss: 0.0013
Epoch 9/20
1087/1087 [==============================] - 10s 9ms/step - loss: 4.4721e-04 - val_loss: 0.0013

<h2>Prediction</h2>

In [ ]:
pred =model.predict(x_test)
fig =plt.figure(facecolor='white')
ax=fig.add_subplot(111)
ax.plot(y_test,labe)